# Combined movies
Craig Lage - 10-Jun-24 \
The goal of this is to create a combined movie of the Simonyi telescope system, including the TMA, M1M3, M2, both hexapods, and the camera rotator.  Also will highlight when exposures are being taken.

In [ ]:
import sys, time, os, yaml, warnings
import shlex, subprocess
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.patches import Polygon, Circle, Rectangle
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
from PIL import Image, ImageOps
from scipy.ndimage import rotate
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable
from lsst_efd_client import EfdClient

## Set up the necessary subroutines

In [ ]:
def heatMapZM1M3(df, ax, FATable, time, zmin, zmax):
    index = df.index.searchsorted(time)
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 Z forces (N)", fontsize=18)

    types = [['SAA','NA', 'o', 'Z'], ['DAA','Y_PLUS', '^', 'Y_PLUS'], ['DAA','Y_MINUS', 'v', 'Y_MINUS'], \
             ['DAA','X_PLUS', '>', 'X_PLUS'], ['DAA','X_MINUS', '<', 'X_MINUS']]

    for [type, orient, marker, label] in types:
        xs = []
        ys = []
        zs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                name=f"zForce{i}"
                zs.append(df.iloc[index][name])# - df_zero.iloc[0][name])
        im = ax.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', vmin=zmin, vmax=zmax, s=50, label=label)
    plt.colorbar(im, ax=ax,fraction=0.055, pad=0.02, cmap='RdBu_r') 
    return
    
def lateralForcesM1M3(df, ax, FATable, time, forceMax):
    index = df.index.searchsorted(time)
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 XY forces (N)", fontsize=18)
    ax.set_xlim(-4.5,4.5)
    ax.set_ylim(-4.5,4.5)
    types = [['DAA','Y_PLUS', '^', 'Y_PLUS','g'], ['DAA','Y_MINUS', 'v', 'Y_MINUS', 'cyan'], \
             ['DAA','X_PLUS', '>', 'X_PLUS', 'r'], ['DAA','X_MINUS', '<', 'X_MINUS', 'r']]
    for [type, orient, marker, label, color] in types:
        xs = []
        ys = []
        arrowXs = []
        arrowYs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                if orient == 'X_PLUS':
                    name = f"xForce{FATable[i].x_index}"
                    arrowXs.append(df.iloc[index][name] / forceMax)
                    arrowYs.append(0.0)
                if orient == 'X_MINUS':
                    name = f"xForce{FATable[i].x_index}"
                    arrowXs.append(-df.iloc[index][name] / forceMax)
                    arrowYs.append(0.0)
                if orient == 'Y_PLUS':
                    name = f"yForce{FATable[i].y_index}"
                    arrowXs.append(0.0)
                    arrowYs.append(df.iloc[index][name] / forceMax)
                if orient == 'Y_MINUS':
                    name = f"yForce{FATable[i].y_index}"
                    arrowXs.append(0.0)
                    arrowYs.append(-df.iloc[index][name] / forceMax)
            else:
                continue
        ax.scatter(xs, ys, marker=marker, color=color, s=50, label=label)
        for ii in range(len(xs)):
            ax.arrow(xs[ii], ys[ii], arrowXs[ii], arrowYs[ii], color=color)

    ax.plot([-4.0,-3.0], [-3.8,-3.8], color='g')
    ax.text(-4.0, -4.3, f"{forceMax} N")
    return

def plotAxialForcesM2(df, yaml_file, ax, zmin, zmax, time):
    # Get the data from the yaml file
    index = df.index.searchsorted(time)
    with open(yaml_file, 'r') as stream:
        locations = yaml.safe_load(stream)
    axials = np.array(locations['locAct_axial'])
    xs = axials[:,0]
    ys = axials[:,1]
    zs = []
    for i in range(len(xs)):
        name=f"measured{i}"
        zs.append(df.iloc[index][name])

    ax.set_xlabel("Y position (m)")
    ax.set_ylabel("X position (m)")
    ax.set_title("M2 forces (N)", fontsize=18)

    im = ax.scatter(ys, xs, marker='o', c=zs, cmap='RdBu_r', vmin=zmin, vmax=zmax, s=80, label="Axial")
    plt.colorbar(im, ax=ax,fraction=0.055, pad=0.02, cmap='RdBu_r')
    return

def plotTangentForcesM2(df, yaml_file, ax, zmax, time):
    index = df.index.searchsorted(time)
    # Get the data from the yaml file
    with open(yaml_file, 'r') as stream:
        locations = yaml.safe_load(stream)
    # Now plot tangential actuator locations
    Rtan = locations['radiusActTangent'] # Radius in meters
    thetas = locations['locAct_tangent']
    width = 0.2
    height = 0.9
    poly = [[-height/2.0, 0],[height/2.0, 0], \
                     [height/2.0, width], [-height/2.0, width]]
    scale = 2.5
    for i, theta in enumerate(thetas):
        theta *= np.pi / 180.0
        rot_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                       [np.sin(theta), np.cos(theta)]]).transpose()
        rot_poly = []
        for point in poly:
            rot_poly.append(np.dot(rot_matrix, point))
        rot_poly = np.array(rot_poly)
        x = Rtan * np.cos(theta)
        y = Rtan * np.sin(theta)
        xy = np.array([y,x]) + rot_poly

        if i == 0:
            ax.add_patch(Polygon(
                xy= xy, linewidth=1, color='coral', \
                fill=True, label='Tangent'))
        else:
            ax.add_patch(Polygon(
                xy=xy, linewidth=1, color='coral', fill=True))
        name=f"measured{i}"
        force =  - df.iloc[index][name]
        length = 0.5 * force / zmax
        dx = length * np.sin(theta)
        dy =  - length * np.cos(theta)
        box_center = np.array([x + np.mean(rot_poly[:,1]), y + np.mean(rot_poly[:,0])])
        arrow_center = box_center + np.array([width * np.cos(theta), width * np.sin(theta)])
        arrow_start = arrow_center - np.array([dx / 2.0, dy / 2.0])       
        ax.arrow(arrow_start[1], arrow_start[0], dy, dx, width=0.02, length_includes_head=True, color='black')
    legend_force = 1000.0
    legend_length = 0.5 * legend_force / zmax
    xs = -scale * 0.9
    ys = scale * 0.85
    dx = legend_length
    dy = 0
    ax.arrow(xs, ys, dx, dy, width=0.02, length_includes_head=True, color='black')
    ax.text(xs + dx * 1.05, ys,  f"{legend_force:.1f} N", color='black')
    ax.set_xlim(-scale, scale)
    ax.set_ylim(-scale, scale)
    return

def plotHexapod(df, ax, time, hex="Camera"):
    index = df.index.searchsorted(time)
    radius = 5.0
    xmin = -8.0; xmax = 8.0
    zmin = -100.0; zmax = 100.0
    X = df.iloc[index]["position0"]
    Y = df.iloc[index]["position1"]
    Z = df.iloc[index]["position2"]
    U = df.iloc[index]["position3"]
    V = df.iloc[index]["position4"]
    # X,Y position
    circle = Circle((X,Y), radius, facecolor='yellow',
                    edgecolor='black',linewidth=3)
    ax.add_patch(circle)
    if hex == 'M2': 
        circle2 = Circle((X,Y), radius * 0.5, facecolor='white',
                edgecolor='black',linewidth=3)
        ax.add_patch(circle2)
    ax.scatter(X,Y, marker='+', s=200, color='black')
    ax.scatter(0.0,0.0, marker='+', s=200, color='black')
    ax.set_xlabel("X (um)")
    ax.set_xticks([-5,0,5])
    ax.set_ylabel("Y (um)")
    ax.set_yticks([-5,0,5])
    ax.set_title(f"{hex} Hexapod", fontsize=18)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(xmin, xmax)
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    # U, V 
    height = 0.2
    mult = 10000.0
    rect = Rectangle((xmin+2.0, xmax-1.0), (xmax - xmin - 4.0), height, facecolor='black',
                angle=mult * U, rotation_point='center')
    ax.add_patch(rect)
    ax.text(xmin+1.0, xmax-2.5, f'U = {U:.1f}"')
    #ax.plot([xmin, xmax], [xmax-1.0, xmax-1.0], ls='--', 
    #         linewidth=1.0, color='black')
    rect = Rectangle((xmax-1.0, xmin+2.0), height, (xmax - xmin - 4.0),  facecolor='black',
                angle=mult * V, rotation_point='center')
    ax.add_patch(rect)
    ax.text(xmax-5.0, xmin+1.0, f'V = {V:.1f}"')
    #ax.plot([xmax-1.0, xmax-1.0], [xmin, xmax], ls='--', 
    #         linewidth=1.0, color='black')

    # Z position
    divider = make_axes_locatable(ax)
    ax1 = divider.append_axes("left", size="20%", pad=0.00)
    fig = ax1.get_figure()
    fig.add_axes(ax1)

    height = 4.0
    rect = Rectangle((0.0, Z - height / 2.0), 1.0, height, facecolor='blue',
                edgecolor='black',linewidth=1)
    ax1.add_patch(rect)
    ax1.set_xlim(0.0,1.0)
    ax1.set_ylim(zmin, zmax)
    ax1.set_xticklabels([])
    ax1.set_ylabel("Z (um)", loc='top', labelpad=-8)
    return

def plotTelescope(az, el, rot, ax, simonyi_file, time):
    # Get the elevation angle at the given time
    el_index = el.index.searchsorted(time)
    ax.set_title("TMA", fontsize=18)
    ax.set_xlim(0, 1.0)
    ax.set_ylim(0, 1.0)
    angle = 90.0 - el['actualPosition'].iloc[el_index]
    tel = Image.open(simonyi_file)   
    tel = ImageOps.expand(tel, border=100, fill=(255,255,255)) 
    rot_tel = rotate(tel, angle, reshape=False)
    #The OffsetBox is a simple container artist.
    #The child artists are meant to be drawn at a relative position to its #parent.
    imagebox = OffsetImage(rot_tel, zoom = 0.15)
    #Annotation box for solar pv logo
    #Container for the imagebox referring to a specific position *xy*.
    #ab = AnnotationBbox(imagebox, (0.0, 0.0), xycoords='axes fraction', box_alignment=(0.5,0.5), frameon = False)
    ab = AnnotationBbox(imagebox, (0.5, 0.8), frameon = False)
    ax.add_artist(ab)
    ax.text(0.8,0.8, "Elevation", color='black')
    ax.set_axis_off()
    #Azimuth
    az_index = az.index.searchsorted(time)
    az_theta = az['actualPosition'].iloc[az_index]%360.0
    az_theta *= np.pi / 180.0
    radius = 0.2
    az_circle = Circle((0.21, 0.3), radius, facecolor='white', 
                    edgecolor='black',linewidth=3)
    ax.add_patch(az_circle)
    dx = radius * np.cos(az_theta)
    dy = radius * np.sin(az_theta)
    ax.arrow(0.21, 0.3, dx, dy, width=0.02, length_includes_head=True, color='black')
    ax.text(0.12, 0.55, "Azimuth", color='black')
    ax.text(0.42, 0.30, "0", color='black', fontsize=8)
    ax.text(0.18, 0.51, "90", color='black', fontsize=8)
    ax.text(-0.08, 0.30, "180", color='black', fontsize=8)
    ax.text(0.18, 0.03, "270", color='black', fontsize=8)
    #Rotator
    rot_index = rot.index.searchsorted(time)
    rot_theta = rot['actualPosition'].iloc[rot_index]
    rot_theta *= np.pi / 180.0
    rot_circle = Circle((0.79, 0.3), radius, facecolor='white', 
                    edgecolor='blue',linewidth=3)
    ax.add_patch(rot_circle)
    dx = radius * np.cos(rot_theta)
    dy = radius * np.sin(rot_theta)
    ax.arrow(0.79, 0.3, dx, dy, width=0.02, length_includes_head=True, color='blue')
    ax.text(0.70, 0.55, "Rotator", color='blue')
    ax.text(1.00, 0.30, "0", color='blue', fontsize=8)
    ax.text(0.76, 0.51, "90", color='blue', fontsize=8)
    ax.text(0.50, 0.30, "180", color='blue', fontsize=8)
    ax.text(0.76, 0.03, "270", color='blue', fontsize=8)
    return

def addExposures(tel, ax, time):
    time = Time(time, scale='utc').unix_tai
    for i in range(len(tel)):
        if time > Time(tel.index[i], scale='utc').unix_tai \
            and time < Time(tel.iloc[i]['close'], scale='utc').unix_tai:
            seqNum = tel.iloc[i]['imageNumber']
            rect = Rectangle((-0.2, 0.65), 0.3, 0.3, facecolor='lime',
            edgecolor='black',linewidth=2, clip_on=False)
            ax.add_patch(rect)
            ax.text(-0.18, 0.70, f"Exposing\nseqNum\n{seqNum}")
            return
        
    

## Define the times and options

In [ ]:
# Times to start looking at EFD values
start = Time("2024-01-10T00:02:00", scale='utc')
end = Time("2024-01-10T00:03:00", scale='utc')
start = Time("2024-01-06T02:45:00", scale='utc')
end = Time("2024-01-06T03:15:00", scale='utc')
# These are a fudge needed because we weren't taking images at the time of the others.
cam_start = Time("2024-05-02T03:00:00", scale='utc')
cam_end = Time("2024-05-02T03:30:00", scale='utc')
cam_delta = cam_start.unix_tai - start.unix_tai
M1M3_zmin = 0.0
M1M3_zmax = 2000.0
M1M3_lateralMax = 1500.0

M2_yaml_file = '../../MTM2/cell_geom/cell_geom.yaml'
M2_simonyi_file = '../../MTM2/cell_geom/Simonyi.png'

M2_axial_zmax = 250
M2_tangent_zmax = 2500


# The following allows you to plot only every nth data point
# If this value is 1, a frame will be made for every data point
# Of course, this takes longer
frameN = 10

# This speeds up the M1M3 queries by only getting the necessary items
# About 2X faster with this.
M1M3_names = []
for i in range(len(FATable)):
    name=f"zForce{i}"
    M1M3_names.append(name)
for i in range(len(FATable)):
    if FATable[i].actuator_type.name == 'DAA':
        if FATable[i].orientation.name in ['X_PLUS', 'X_MINUS']:
            name = f"xForce{FATable[i].x_index}"
            M1M3_names.append(name)
        if FATable[i].orientation.name in ['Y_PLUS', 'Y_MINUS']:
            name = f"yForce{FATable[i].y_index}"
            M1M3_names.append(name)

## Now generate the frames
### This will take some time
### First, pull the data from the EFD

In [ ]:
client = EfdClient('usdf_efd')
timestamp = start.isot.split('.')[0].replace('-','').replace(':','')
dirName = f"/home/c/cslage/u/MTAOS/movies/movie_{timestamp}"
%mkdir -p {dirName}
movieName = f"Combined_movie_{timestamp}.mp4"
print(movieName)
#Get the data from the EFD
el = await client.select_time_series("lsst.sal.MTMount.elevation", \
                                            ['actualPosition'], \
                                             start, end)
print(len(el))
az = await client.select_time_series("lsst.sal.MTMount.azimuth", \
                                            ['actualPosition'], \
                                             start, end)
print(len(az))
rot = await client.select_time_series("lsst.sal.MTRotator.rotation", \
                                            ['actualPosition'], \
                                             start, end)
print(len(rot))

M1M3_forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", M1M3_names, \
                                         start, end)
print(len(M1M3_forces))
M2_axial_forces = await client.select_time_series("lsst.sal.MTM2.axialForce", "*", \
                                             start, end)
print(len(M2_axial_forces))
M2_tangent_forces = await client.select_time_series("lsst.sal.MTM2.tangentForce", "*", \
                                             start, end)
print(len(M2_tangent_forces))
camhex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=1)
print(len(camhex))
m2hex = await client.select_time_series('lsst.sal.MTHexapod.application', ['*'], start, end, index=2)
print(len(m2hex))


tel = await client.select_time_series("lsst.sal.CCCamera.logevent_endOfImageTelemetry", \
                                            ['imageNumber', 'timestampAcquisitionStart', 'exposureTime'], \
                                             cam_start, cam_end)

# This line is a fudge to shift the exposures to line up with the rest.
tel['timestampAcquisitionStart'] = tel['timestampAcquisitionStart'] - cam_delta
tel['close'] = tel['timestampAcquisitionStart'] + tel['exposureTime']
tel['close'] = Time(tel['close'], format='unix_tai', scale='tai').utc.isot
tel['timestampAcquisitionStart'] = Time(tel['timestampAcquisitionStart'], format='unix_tai', scale='tai').utc.isot
tel = tel.set_index('timestampAcquisitionStart')
print(len(tel))


### Now build the frames

In [ ]:
# Build the individual frames
fig = plt.figure(figsize=(12, 8))
plt.subplots_adjust(wspace=0.4, hspace=0.2)
for n in range(0, len(el), frameN):
    try:
        time = el.index[n]
        ax1 = fig.add_subplot(2,3,1, aspect=1.0)
        lateralForcesM1M3(M1M3_forces, ax1, FATable, time, M1M3_lateralMax)
        ax2 = fig.add_subplot(2,3,2, aspect=1.0)
        heatMapZM1M3(M1M3_forces, ax2, FATable, time, M1M3_zmin, M1M3_zmax)
        ax3 = fig.add_subplot(2,3,3, aspect=1.0)
        plotAxialForcesM2(M2_axial_forces, M2_yaml_file, ax3, 0, M2_axial_zmax, time)
        plotTangentForcesM2(M2_tangent_forces, M2_yaml_file, ax3, M2_tangent_zmax, time)
        ax4 = fig.add_subplot(2,3,4, aspect=1.0)
        plotHexapod(camhex, ax4, time, hex="Camera") 
        ax5 = fig.add_subplot(2,3,5, aspect=1.0)
        plotHexapod(m2hex, ax5, time, hex="M2")
        ax6 = fig.add_subplot(2,3,6, aspect=1.0)
        plotTelescope(az, el, rot, ax6, M2_simonyi_file, time)
        addExposures(tel, ax6, time)
        plt.suptitle(f"Simonyi telescope movie {time}", fontsize=18)
        plt.savefig(f"{dirName}/Frame_{n:05d}.png")
        plt.clf()
    except:
        continue


## Now build the movie

In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()